# Visualización de Resultados de Modelos

Este notebook permite cargar y visualizar los resultados de entrenamiento de los modelos de clasificación de estadios de sueño.

Carga métricas desde archivos JSON generados por `src/models.py` y genera visualizaciones interactivas.


## Configuración inicial


In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
from src.models import load_metrics

sns.set_theme(style="whitegrid", palette="deep")

PROJECT_ROOT = Path("..").resolve()
MODELS_DIR = PROJECT_ROOT / "models"

STAGE_ORDER = ["W", "N1", "N2", "N3", "REM"]
STAGE_COLORS = {
    "W": "#fdae61",
    "N1": "#fee090",
    "N2": "#abd9e9",
    "N3": "#2c7bb6",
    "REM": "#d7191c",
}

## Cargar métricas de un modelo


In [ ]:
# Cambiar según el modelo que quieras visualizar
model_type = "random_forest"  # o "xgboost"
metrics_path = MODELS_DIR / f"{model_type}_metrics.json"

if metrics_path.exists():
    metrics = load_metrics(metrics_path)
    print(f"Métricas cargadas para {model_type}")
    print(f"Timestamp: {metrics.get('timestamp', 'N/A')}")
    print(f"Parámetros del modelo: {metrics.get('model_params', {})}")
else:
    print(f"No se encontró {metrics_path}")
    print("Ejecuta primero: python src/models.py --model-type random_forest")

## Métricas generales


In [ ]:
if "metrics" in metrics:
    m = metrics["metrics"]
    print("=" * 60)
    print("MÉTRICAS GENERALES")
    print("=" * 60)
    print(f"Accuracy:        {m.get('accuracy', 0):.4f}")
    print(f"Cohen's Kappa:   {m.get('kappa', 0):.4f}")
    print(f"F1-score (macro): {m.get('f1_macro', 0):.4f}")
    print(f"F1-score (weighted): {m.get('f1_weighted', 0):.4f}")

    if "cv_results" in metrics:
        cv = metrics["cv_results"]
        print("\nCross-Validation:")
        print(
            f"  Mean F1-macro: {cv.get('mean_score', 0):.4f} ± {cv.get('std_score', 0):.4f}"
        )

## F1-score por estadio


In [ ]:
if "metrics" in metrics and "f1_per_class" in metrics["metrics"]:
    f1_per_class = metrics["metrics"]["f1_per_class"]

    # Crear DataFrame para visualización
    f1_df = pd.DataFrame(
        [
            {"Stage": stage, "F1-score": f1_per_class.get(stage, 0.0)}
            for stage in STAGE_ORDER
        ]
    )

    # Gráfico de barras
    fig, ax = plt.subplots(figsize=(10, 6))
    colors = [STAGE_COLORS.get(stage, "gray") for stage in STAGE_ORDER]
    bars = ax.bar(
        f1_df["Stage"], f1_df["F1-score"], color=colors, alpha=0.7, edgecolor="black"
    )

    # Agregar valores en las barras
    for bar in bars:
        height = bar.get_height()
        ax.text(
            bar.get_x() + bar.get_width() / 2.0,
            height,
            f"{height:.3f}",
            ha="center",
            va="bottom",
            fontweight="bold",
        )

    ax.set_ylabel("F1-score", fontsize=12)
    ax.set_xlabel("Estadio de Sueño", fontsize=12)
    ax.set_title(
        f"F1-score por Estadio - {model_type.upper()}", fontsize=14, fontweight="bold"
    )
    ax.set_ylim([0, 1.1])
    ax.grid(axis="y", alpha=0.3)

    plt.tight_layout()
    plt.show()

    # Mostrar tabla
    display(
        f1_df.style.format({"F1-score": "{:.4f}"}).background_gradient(
            cmap="YlGnBu", subset=["F1-score"]
        )
    )

## Matriz de confusión


In [ ]:
if "confusion_matrix" in metrics:
    cm = np.array(metrics["confusion_matrix"])

    # Normalizar por filas (recall por clase)
    cm_normalized = cm.astype("float") / (cm.sum(axis=1)[:, np.newaxis] + 1e-10)

    fig, axes = plt.subplots(1, 2, figsize=(16, 6))

    # Matriz absoluta
    sns.heatmap(
        cm,
        annot=True,
        fmt="d",
        cmap="Blues",
        xticklabels=STAGE_ORDER,
        yticklabels=STAGE_ORDER,
        ax=axes[0],
        cbar_kws={"label": "Count"},
    )
    axes[0].set_title("Matriz de Confusión (Absoluta)", fontsize=14, fontweight="bold")
    axes[0].set_ylabel("Verdadero", fontsize=12)
    axes[0].set_xlabel("Predicho", fontsize=12)

    # Matriz normalizada
    sns.heatmap(
        cm_normalized,
        annot=True,
        fmt=".3f",
        cmap="Blues",
        xticklabels=STAGE_ORDER,
        yticklabels=STAGE_ORDER,
        ax=axes[1],
        cbar_kws={"label": "Proporción"},
        vmin=0,
        vmax=1,
    )
    axes[1].set_title(
        "Matriz de Confusión (Normalizada)", fontsize=14, fontweight="bold"
    )
    axes[1].set_ylabel("Verdadero", fontsize=12)
    axes[1].set_xlabel("Predicho", fontsize=12)

    plt.tight_layout()
    plt.show()

## Comparar múltiples modelos


In [ ]:
# Cargar métricas de todos los modelos disponibles
all_metrics = {}
for model_type in ["random_forest", "xgboost"]:
    metrics_path = MODELS_DIR / f"{model_type}_metrics.json"
    if metrics_path.exists():
        all_metrics[model_type] = load_metrics(metrics_path)

if len(all_metrics) > 1:
    # Comparar métricas generales
    comparison = []
    for name, m in all_metrics.items():
        if "metrics" in m:
            comparison.append(
                {
                    "Modelo": name,
                    "Accuracy": m["metrics"].get("accuracy", 0),
                    "Kappa": m["metrics"].get("kappa", 0),
                    "F1-macro": m["metrics"].get("f1_macro", 0),
                    "F1-weighted": m["metrics"].get("f1_weighted", 0),
                }
            )

    if comparison:
        df_comp = pd.DataFrame(comparison)
        display(
            df_comp.style.format(
                {
                    "Accuracy": "{:.4f}",
                    "Kappa": "{:.4f}",
                    "F1-macro": "{:.4f}",
                    "F1-weighted": "{:.4f}",
                }
            ).background_gradient(
                cmap="RdYlGn", subset=["Accuracy", "Kappa", "F1-macro", "F1-weighted"]
            )
        )

        # Gráfico comparativo
        fig, ax = plt.subplots(figsize=(12, 6))
        x = np.arange(len(df_comp))
        width = 0.2

        metrics_to_plot = ["Accuracy", "Kappa", "F1-macro", "F1-weighted"]
        for i, metric in enumerate(metrics_to_plot):
            ax.bar(x + i * width, df_comp[metric], width, label=metric, alpha=0.8)

        ax.set_ylabel("Score", fontsize=12)
        ax.set_title("Comparación de Modelos", fontsize=14, fontweight="bold")
        ax.set_xticks(x + width * 1.5)
        ax.set_xticklabels(df_comp["Modelo"])
        ax.legend()
        ax.grid(axis="y", alpha=0.3)
        ax.set_ylim([0, 1])

        plt.tight_layout()
        plt.show()
else:
    print("Necesitas al menos 2 modelos entrenados para comparar")